In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
import pyspark.sql.functions as F

conf = SparkConf().setAppName("1731").setMaster("local[4]")
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/29 01:34:11 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.102 instead (on interface enp0s3)
25/07/29 01:34:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/29 01:34:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/29 01:34:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
'''
Table: Employees

+-------------+----------+
| Column Name | Type     |
+-------------+----------+
| employee_id | int      |
| name        | varchar  |
| reports_to  | int      |
| age         | int      |
+-------------+----------+
employee_id is the column with unique values for this table.
This table contains information about the employees and the id of the manager they report to. 
Some employees do not report to anyone (reports_to is null). 
 

For this problem, we will consider a manager an employee who has at least 1 other employee 
reporting to them.

Write a solution to report the ids and the names of all managers, the number of employees 
who report directly to them, and the average age of the reports rounded to the nearest integer.

Return the result table ordered by employee_id.

The result format is in the following example.

 

Example 1:

Input: 
Employees table:
+-------------+---------+------------+-----+
| employee_id | name    | reports_to | age |
+-------------+---------+------------+-----+
| 9           | Hercy   | null       | 43  |
| 6           | Alice   | 9          | 41  |
| 4           | Bob     | 9          | 36  |
| 2           | Winston | null       | 37  |
+-------------+---------+------------+-----+
Output: 
+-------------+-------+---------------+-------------+
| employee_id | name  | reports_count | average_age |
+-------------+-------+---------------+-------------+
| 9           | Hercy | 2             | 39          |
+-------------+-------+---------------+-------------+
Explanation: Hercy has 2 people report directly to him, Alice and Bob. 
Their average age is (41+36)/2 = 38.5, which is 39 after rounding it to the nearest integer.
'''

In [3]:
data = [
(9,'Hercy'  ,None,43),
(6,'Alice'  ,9   ,41),
(4,'Bob'    ,9   ,36),
(2,'Winston',None,37)    
]
schema = ['employee_id','name','reports_to','age']

In [5]:
df = spark.createDataFrame(data = data, schema = schema)
df.show()

+-----------+-------+----------+---+
|employee_id|   name|reports_to|age|
+-----------+-------+----------+---+
|          9|  Hercy|      NULL| 43|
|          6|  Alice|         9| 41|
|          4|    Bob|         9| 36|
|          2|Winston|      NULL| 37|
+-----------+-------+----------+---+



In [9]:
df.alias("e").join(df.alias("m"),
                   F.col("e.reports_to") == F.col("m.employee_id"),
                   'inner'
                  )\
             .groupBy(F.col("m.employee_id"),F.col("m.name"))\
             .agg(F.count(F.col("e.employee_id")).alias("reports_count"),
                  F.round(F.avg(F.col("e.age")),0).alias("average_age")
                 )\
             .show()

+-----------+-----+-------------+-----------+
|employee_id| name|reports_count|average_age|
+-----------+-----+-------------+-----------+
|          9|Hercy|            2|       39.0|
+-----------+-----+-------------+-----------+



## SQL Solution 
<pre>
SELECT m.employee_id, m.name, COUNT(e.employee_id) as reports_count, ROUND(AVG(e.age),0) as average_age
FROM Employees e
LEFT JOIN Employees m ON m.employee_id = e.reports_to
GROUP BY m.employee_id, m.name
</pre>